# Extracting MX spot prices from PDF

As part of their monthly reporting procedure,Master Data Management team extracts prices of raw materials by copy pasting them from the downloaded PDFs into their existing Excel files that store price data.

Prices for the current period are extracted,as well as the ones for 3 future periods.

To minimize the need of this manual data entry procedure,this script reads the data,indexes the required row and applies data transformation techniques to prepare the data for export to Excel.

# Introduction

This notebook is divided into several chapters: Data Loading,Data Preparation and Writing Data to an Excel workbook.The goal of the first one is to read the data from the PDF and select only the rows required.

In the Data Preparation section,using date functions,the headers will be changed to reflect the current and future periods.
As a last step,the extracted and transformed data is written to the respective Excel workbook.

## Data loading

### Import required libraries

In [31]:
from tabula.io import read_pdf
import pandas as pd
import numpy as np
import openpyxl 
import datetime
from dateutil import relativedelta
import os
import calendar
import datetime

### Read the data

In [32]:
df = read_pdf("C:\\Users\\user\\Downloads\\Argus Toluene and Xylenes Outlook (2024-06-26).pdf", pages="all")

In [33]:
df[0]

,US,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,Jun 17,Jul 17,Aug 17,Sep 17,Oct 17,Nov 17,Dec 17,Jan 18,Feb 18,Mar 18,Apr 18,May 18,Jun 18
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tol spot ¢/USG,202,193,191,199,202,210,213,206,210,201,207,206,213
3,Tol spot $/t,614,586,580,605,614,638,648,624,638,612,630,627,647
4,MX spot ¢/USG,209,203,201,214,215,221,217,210,213,202,211,220,231
5,MX spot $/t,636,619,611,653,656,674,661,641,650,616,644,669,703
6,OX contract ¢/lb,39,38,37,37,39,39,38,37,36,36,34,35,36
7,PX contract ¢/lb,41,39,39,40,40,42,41,40,40,39,39,40,41
8,PTA contract ¢/lb,42,41,41,42,42,42,42,41,41,41,41,41,42
9,MEG contract ¢/lb,42,41,36,35,35,35,34,34,34,33,33,34,33


### Select the required row of data

In [34]:
df_new=df[0][26:30].iloc[3:]

### Display data

In [35]:
df_new

,US,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
29,MX spot $/t,625,606,603,619,634,623,610,609,587,579,570,581,574


## Data preparation

The goal of the Data reparation module is to prepare the Actual and Forecast prices for export the Excel.
That is achieved by extracting the current and forecast month and year and concatenating them.Afterwards,the header names are changed respectively.

### Get month and year (actual and forecast

In [36]:
month =datetime.date.today()-relativedelta.relativedelta(months=1)
year = str(datetime.date.today().year)
strMonth=str(month.month)

In [37]:
currentdate= "-".join([strMonth,year])

In [38]:
currentdate

'5-2024'

In [39]:
nextmonth1 = datetime.date.today()

In [40]:
nextmonth=str(nextmonth1.month)

In [41]:
nextmonthyear=str(nextmonth1.year)

In [42]:
concatenatedcurrent="-".join([nextmonth, nextmonthyear])

In [43]:
secondmonth1=datetime.date.today()+ relativedelta.relativedelta(months=1)

In [44]:
secondmonth=str(secondmonth1.month)

In [45]:
secondmonthyear=str(secondmonth1.year)

In [46]:
concatenatednext="-".join([secondmonth, secondmonthyear])

In [47]:
thirdmonth1 =datetime.date.today()+ relativedelta.relativedelta(months=2)

In [48]:
thirdmonth=str(thirdmonth1.month)

In [49]:
thirdmonthyear=str(thirdmonth1.year)

In [50]:
concatenatedlast="-".join([thirdmonth, thirdmonthyear])

In [51]:
concatenatedcurrent

'6-2024'

## Change header names

In [52]:
dict = {'US':'Product',
    'Unnamed: 0':currentdate,
    'Unnamed: 1':concatenatedcurrent,
        'Unnamed: 2':concatenatednext,
        'Unnamed: 3':concatenatedlast}
 
# call rename () method
df_new.rename(columns=dict,
          inplace=True)
 
# print Data frame after rename columns
display(df_new)

,Product,5-2024,6-2024,7-2024,8-2024,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
29,MX spot $/t,625,606,603,619,634,623,610,609,587,579,570,581,574


In [53]:
df_final=df_new.melt(id_vars=['Product'], var_name='Date', value_name='Price').iloc[0:4]

In [54]:
df_current=df_final.iloc[:1]

In [55]:
df_forecast=df_final.iloc[1:]

In [56]:
df_forecast

,Product,Date,Price
1,MX spot $/t,6-2024,606
2,MX spot $/t,7-2024,603
3,MX spot $/t,8-2024,619


In [57]:
df_current

,Product,Date,Price
0,MX spot $/t,5-2024,625


In [58]:
df_current2=df_current.values.tolist()

### Data writing to Excel workbook

In [59]:
book = openpyxl.load_workbook("C:\\Users\\user\\Downloads\\toluene1.xlsx")
writer = pd.ExcelWriter("C:\\Users\\user\\Downloads\\toluene1.xlsx", engine='openpyxl')
writer.book = book
writer.sheets = {ws.title: ws for ws in book.worksheets}

ws_current=book["Actual Toluene"]
for sheetname in writer.sheets:
    df_forecast.to_excel(writer,sheet_name="Forecast Toluene", index = False)
    print("Forecast prices written successfully to Excel file")
    
for i in range(len(df_current2)):
    ws_current.append(df_current2[i])
    print("Actual prices written successfully to Excel file")

Forecast prices written successfully to Excel file
Forecast prices written successfully to Excel file
Actual prices written successfully to Excel file


In [60]:
writer.close()